In [ ]:
import csv
import pprint
import pandas as pd
djdata = list(csv.DictReader(open('DJI From 2004.csv')))
vixdata = list(csv.DictReader(open('vixcurrent.csv')))

djchange = [row['Change'] for row in djdata]
vixchange = [row['Change'] for row in vixdata]

djch = [float(i) for i in djchange]
vixch = [float(i) for i in vixchange]


print(len(djch))
print(len(vixch))


In [ ]:
import math

####
#
# LINEAR ALGEBRA
#
####

def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

####
#
# STATS
#
####

def mean(x):
    return sum(x) / len(x)

def de_mean(x):
    """translate x by subtracting its mean (so the result has mean 0)"""
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

def variance(x):
    """assumes x has at least two elements"""
    n = len(x)
    deviations = de_mean(x)
    return sum_of_squares(deviations) / (n - 1)

def standard_deviation(x):
    return math.sqrt(variance(x))

####
#
# CORRELATION
#
#####

def covariance(x, y):
    n = len(x)
    return dot(de_mean(x), de_mean(y)) / (n - 1)

def correlation(x, y):
    stdev_x = standard_deviation(x)
    stdev_y = standard_deviation(y)
    if stdev_x > 0 and stdev_y > 0:
        return covariance(x, y) / (stdev_x * stdev_y)
    else:
        return 0 # if no variation, correlation is zero

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

print('\nScatter plot for daily minutes vs number of friends:')
_ = plt.scatter(vixch, djch)
_ = plt.xlim(-0.5,0.5)
_ = plt.ylim(-0.5,0.5)
_ = plt.xlabel('The change of Vix')
_ = plt.ylabel('The change of DJ')



In [ ]:
print('Correlation before outlier removal: {:.3f}'.format(correlation(djch, vixch)))


In [ ]:
def predict(alpha, beta, x_i):
    """the linear model
    alpha and beta are our model parameters
    x_i is the data point
    return the predicted y value for x_i"""
    return beta * x_i + alpha

def error(alpha, beta, x_i, y_i):
    """the difference between the true y_i
    and our predicted y value"""
    return y_i - predict(alpha, beta, x_i)

def sum_of_squared_errors(alpha, beta, x, y):
    return sum(error(alpha, beta, x_i, y_i) ** 2
               for x_i, y_i in zip(x, y))

def least_squares_fit(x,y):
    """given training values for x and y,
    find the least-squares values of alpha and beta"""
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

def total_sum_of_squares(y):
    """the total squared variation of y_i's from their mean"""
    return sum(v ** 2 for v in de_mean(y))

def r_squared(alpha, beta, x, y):
    """the fraction of variation in y captured by the model, which equals
    1 - the fraction of variation in y not captured by the model"""
    return 1.0 - (sum_of_squared_errors(alpha, beta, x, y) /
                  total_sum_of_squares(y))

alpha, beta = least_squares_fit(vixch, djch)
print("alpha", alpha)
print("beta", beta)

r_squared_value = r_squared(alpha, beta, vixch, djch)
print("r-squared", r_squared_value)

In [ ]:
def standard_error(alpha, beta, x, y):
    return math.sqrt(sum_of_squared_errors(alpha, beta, x, y) / len(y))

stderr = standard_error(alpha, beta, vixch, djch)
print("standard error", stderr)


print('\nScatter plot with regression line and 95% prediction interval:')
_ = plt.scatter(vixch, djch)
_ = plt.xlim(-0.5,0.5)
_ = plt.ylim(-0.2,0.2)
_ = plt.xlabel('The change of Vix')
_ = plt.ylabel('The change of DJ')
_ = plt.plot([-10,10], [predict(alpha,beta,-10), predict(alpha,beta,10)], 'r',
             [-10,10], [predict(alpha,beta,-10)-2*stderr, predict(alpha,beta,10)-2*stderr], 'r--',
             [-10,10], [predict(alpha,beta,-10)+2*stderr, predict(alpha,beta,10)+2*stderr], 'r--')



In [ ]:
# 1 - 
from scipy.stats import linregress
slope, intercept, r_value, p_value, slope_stderr = linregress(vixch,djch)

print('\nComparing linear regression values:')
print('-'*47)
print('{:15} {:>15} {:>15}'.format('Statistic', 'From Scipy', 'From Scratch'))
print('-'*47)
for name, sc_val, sp_val in [("alpha", intercept, alpha),
                             ("beta", slope, beta),
                             ("r-squared", r_squared_value, r_value**2)]:
    print('{:15} {:15.2f} {:15.2f}'.format(name, sc_val, sp_val))
print('-'*47)

# 2 - 
print('\np-value for a two-sided test of H0 that slope is 0: {}'.format(p_value))
print('Can we reject H0?', 'Yes' if p_value<=0.01 else 'No')


In [ ]:
arrv = np.array(vixch)
arrd = np.array(djch)

vixchr = arrv.reshape(-1,1)
djchr = arrd.reshape(-1,1)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math
import numpy as np
from sklearn.metrics import r2_score


# First let's create a train and test split
X_train, X_test, Y_train, Y_test = train_test_split(vixchr, djchr, test_size=0.33,
                                                    random_state=2) # so we get the same results

# Now let's fit a model
lm = LinearRegression()
_ = lm.fit(X_train, Y_train)


print('Intercept:', lm.intercept_)
print('Coefficients:\n', lm.coef_)
print('\nR-squared:', lm.score(X_train, Y_train))

# We use the score method to get r-squared
print('\nR-squared:', lm.score(X_train, Y_train))

# We can predict the median price for a new neighbourhoods
print('\nPredicted price of first five neighbourhoods from test split:\n', lm.predict(X_test)[:5])

# We use the score method to get r-squared
print('\nR-squared:', lm.score(X_train, Y_train))

# We can also calculate the standard error
stderr = math.sqrt(np.mean((Y_train - lm.predict(X_train))**2))
print('\nStandard error:', stderr)

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
X = vixchr
y = djchr

est = sm.OLS(y, X)
est2 = est.fit()
print(est2.summary())

In [ ]:
# 1 - Yes, r_squared indicates that our model explains the data reasonable well.
#     But we should look at standard error as well (page 183 of Data Science from Scratch).

# 2 - 
from sklearn.metrics import mean_squared_error
se1 = math.sqrt(np.mean((Y_train - lm.predict(X_train))**2))
print('Standard error:', se1)
se2 = math.sqrt(mean_squared_error(Y_train, lm.predict(X_train)))
print('Standard error using sklearn.metrics:', se2)

# 3 - Note we have a fairly small data set (339 in training, 167 in test).
#     So this value will vary depending on our split. 
print('\nPredictions should be within {:.1f}k dollars at 95% confidence according to training data.'.format(2*se2))
se_test = math.sqrt(mean_squared_error(Y_test, lm.predict(X_test)))
print('Predictions should be within {:.1f}k dollars at 95% confidence according to test data.'.format(2*se_test))

# 4 - Yes the linear model is appropriate
print('\nResidual plot for training data (blue) and test data (green):')
_ = plt.scatter(lm.predict(X_train), Y_train-lm.predict(X_train), c='blue', s=40, alpha=0.5, edgecolor='white')
_ = plt.scatter(lm.predict(X_test), Y_test-lm.predict(X_test), c='green', s=40, alpha=0.5, edgecolor='white')
_ = plt.plot([-0.1,0.1], [0,0], c='black')
_ = plt.ylabel('Residuals ($y - \hat{y}$)')
_ = plt.xlabel('Predicted values ($\hat{y}$)')

In [ ]:
import csv
import pprint
import pandas as pd
import itertools
import numpy as np

djdata = list(csv.DictReader(open('DJI From 2004v.csv')))
vixdata = list(csv.DictReader(open('vixcurrent.csv')))

djch = [row['trend'] for row in djdata]
djvlm = [row['Volume'] for row in djdata]
vixchange = [row['vixchgystd'] for row in djdata]
vixcha = [float(i) for i in vixchange]

a=list(itertools.chain.from_iterable(zip(vixcha,djvlm)))
arr = np.array(a)
vixvlm= arr.reshape(-1,2)
pprint.pprint((vixvlm))


In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pprint
iris = load_iris()
key='-1=down, 0 = horizontal ,1=up '
#pprint.pprint(key)
#pprint.pprint(iris.data)
#pprint.pprint(iris.target)
#print(len(iris.target))
#print(len(iris.data))

# First let's create a train and test split
X_train, X_test, Y_train, Y_test = train_test_split(vixvlm, djch, test_size=0.33,
                                                    random_state=5) # so we get the same results


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
# Let's fit a model


tree = DecisionTreeClassifier(max_depth=2)
_ = tree.fit(X_train, Y_train)

# Evaluate
print('Classification report ({}):\n'.format(key))
print(classification_report(Y_test, tree.predict(X_test)))

In [ ]:
#From http://chrisstrelioff.ws/sandbox/2015/06/08/decision_trees_in_python_with_scikit_learn_and_pandas.html
def get_code(tree, feature_names, target_names, spacer_base="    "):
    """Produce psuedo-code for decision tree.

    Args
    ----
    tree -- scikit-leant DescisionTree.
    feature_names -- list of feature names.
    target_names -- list of target (class) names.
    spacer_base -- used for spacing code (default: "    ").

    Notes
    -----
    based on http://stackoverflow.com/a/30104792.
    """
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value

    def recurse(left, right, threshold, features, node, depth):
        spacer = spacer_base * depth
        if (threshold[node] != -2):
            print(spacer + "if ( " + features[node] + " <= " + \
                  str(threshold[node]) + " ) {")
            if left[node] != -1:
                    recurse(left, right, threshold, features,
                            left[node], depth+1)
            print(spacer + "}\n" + spacer +"else {")
            if right[node] != -1:
                    recurse(left, right, threshold, features,
                            right[node], depth+1)
            print(spacer + "}")
        else:
            target = value[node]
            for i, v in zip(np.nonzero(target)[1],
                            target[np.nonzero(target)]):
                target_name = target_names[i]
                target_count = int(v)
                print(spacer + "return " + str(target_name) + \
                      " ( " + str(target_count) + " examples )")

    recurse(left, right, threshold, features, 0, 0)
    
print('Decision tree:\n')
get_code(tree, ['vixchange','volume'], ['-1','0','1'])



In [ ]:
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(tree,out_file='tree.dot',
                feature_names = ['vixchange','volume'],
                class_names =['-1','0','1'],
                rounded = True, proportion =False,
                precision = 2, filled = True)



In [ ]:
from scipy.stats import chi2

def mcnemar(x, y):
    n1 = np.sum(x < y)
    n2 = np.sum(x > y)
    stat = (np.abs(n1-n2)-1)**2 / (n1+n2)
    df = 1
    pval = chi2.sf(stat,1)
    return stat, pval

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Perform grid search
param_grid = [
    {'max_depth': [2, 3, 4, 5, 6],
     'criterion': ['entropy', 'gini'],
     'splitter': ['best', 'random']}
]
tree = GridSearchCV(DecisionTreeClassifier(), param_grid)
tree.fit(X_train, Y_train)

# Print grid search results
print('Grid search mean and stdev:\n')
scoring = tree.cv_results_
for mean_score, std, params in zip(scoring['mean_test_score'],scoring['std_test_score'],scoring['params']):
    print("{:0.3f} (+/-{:0.03f}) for {}".format(
            mean_score, std * 2, params))
    
#for params, mean_score, scores in tree.cv_results_:
   # print("{:0.3f} (+/-{:0.03f}) for {}".format(
       #     mean_score, scores.std() * 2, params))

# Print best params
print('\nBest parameters:', tree.best_params_)

# Evaluate on held-out test
print('\nClassification report ({}):\n'.format(key))
print(classification_report(Y_test, tree.predict(X_test)))